#Text to SQL with smolagents

In [1]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
# Agente Industrial Inteligente com Smolagents
# Adaptado para banco de dados de manutenção industrial

# Importação de bibliotecas necessárias
import os

from sqlalchemy import create_engine, inspect, text
from smolagents import tool, CodeAgent, InferenceClientModel



!wget -O manutencao_industrial.db "https://drive.google.com/uc?export=download&id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v"


# Incluir seu HF_TOKEN diretamente:
os.environ["HF_TOKEN"] = " "

--2025-07-01 07:33:09--  https://drive.google.com/uc?export=download&id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v
Resolving drive.google.com (drive.google.com)... 74.125.23.138, 74.125.23.100, 74.125.23.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v&export=download [following]
--2025-07-01 07:33:10--  https://drive.usercontent.google.com/download?id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.174.132, 2404:6800:4008:c1b::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.174.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24576 (24K) [application/octet-stream]
Saving to: ‘manutencao_industrial.db’

manutencao_industri 100%[===================>]  24.00K  --.

In [10]:
class IndustrialMaintenanceAgent:
    def __init__(self, db_path="/content/manutencao_industrial.db"):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}")
        self.setup_sql_tool()
        self.create_agent()

    def analyze_database_structure(self):
        """Analisa a estrutura do banco de dados e retorna uma descrição detalhada"""
        inspector = inspect(self.engine)
        tables = inspector.get_table_names()

        description = "Banco de dados de manutenção industrial. Tabelas disponíveis:\n\n"

        for table in tables:
            columns_info = inspector.get_columns(table)
            foreign_keys = inspector.get_foreign_keys(table)

            description += f"Tabela '{table}':\n"
            description += "Colunas:\n"

            for col in columns_info:
                col_desc = f" - {col['name']}: {col['type']}"
                if not col['nullable']:
                    col_desc += " (NOT NULL)"
                if col.get('primary_key'):
                    col_desc += " (CHAVE PRIMÁRIA)"
                description += col_desc + "\n"

            if foreign_keys:
                description += "Relações:\n"
                for fk in foreign_keys:
                    description += f" - {fk['constrained_columns']} → {fk['referred_table']}.{fk['referred_columns']}\n"

            description += "\n"

        return description

    def setup_sql_tool(self):
        """Configura a ferramenta SQL com a descrição do banco de dados"""
        db_description = self.analyze_database_structure()

        @tool
        def sql_maintenance_query(query: str) -> str:
            """
            Permite realizar consultas SQL no banco de dados de manutenção industrial.
            Retorna uma string com os resultados.

            O banco de dados contém informações sobre manutenção industrial incluindo:
            - Ordens de manutenção com datas e descrições
            - Técnicos com especialidades e turnos de trabalho
            - Equipamentos industriais com tipos e especificações
            - Relações entre ordens, técnicos e equipamentos

            Args:
                query: Consulta SQL a ser executada. Deve ser SQL válido.
            """
            try:
                # Obter descrição do banco para contexto adicional
                db_info = self.analyze_database_structure()

                with self.engine.connect() as con:
                    result = con.execute(text(query))
                    rows = result.fetchall()

                    if not rows:
                        return "Nenhum resultado encontrado."

                    # Formatar resultados de forma legível
                    columns = result.keys()
                    output = "Resultados:\n"
                    output += " | ".join(columns) + "\n"
                    output += "-" * (len(" | ".join(columns))) + "\n"

                    for row in rows:
                        output += " | ".join(str(value) for value in row) + "\n"

                    return output

            except Exception as e:
                return f"Erro na consulta SQL: {str(e)}"

        self.sql_tool = sql_maintenance_query

    def create_agent(self):
        """Cria o agente com o modelo especificado"""
        self.agent = CodeAgent(
            tools=[self.sql_tool],
            model=InferenceClientModel(#"microsoft/Phi-3.5-mini-instruct"
                model_id="Qwen/Qwen2.5-Coder-32B-Instruct"
                #model_id="Qwen/Qwen2-72B-Instruct"
            ),
            #max_steps = 5,
        )

    def query(self, question: str):
        """Processa uma pergunta em linguagem natural"""
        try:
            response = self.agent.run(question)
            return response
        except Exception as e:
            return f"Erro ao processar a pergunta: {str(e)}"

    def get_sample_questions(self):
        """Retorna exemplos de perguntas que o agente pode responder"""
        return [
            "Qual técnico trabalhou na ordem de manutenção número 32?",
            "Quais são os tipos de equipamento que passaram por manutenção nos últimos 3 meses?",
            "Qual é o técnico que trabalhou em mais ordens de manutenção?",
            "Quantas ordens de manutenção foram realizadas neste ano?",
            "Quais equipamentos precisam de manutenção preventiva?",
            "Qual é a especialidade do técnico que mais trabalhou?",
            "Em qual turno ocorrem mais manutenções?",
            "Qual foi a última ordem de manutenção realizada?"
        ]

In [11]:
agent = IndustrialMaintenanceAgent()
question = agent.get_sample_questions()[0]
response = agent.query(question)
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Qual técnico trabalhou na ordem de manutenção número 32?                                                        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query_result = sql_maintenance_query(query="SELECT tecnico FROM ordens_de_manutencao WHERE numero_ordem = 32")   
  print("Technician for maintenance order 32:", query_result)                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Technician for maintenance order 32: Erro na consulta SQL: (sqlite3.OperationalError) no such table: 
ordens_de_manutencao
[SQL: SELECT tecnico FROM ordens_de_manutencao WHERE numero_ordem = 32]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Out: None

[Step 1: Duration 1.54 seconds| Input tokens: 2,080 | Output tokens: 99]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Tecnico 3")                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Tecnico 3

[Step 8: Duration 1.27 seconds| Input tokens: 27,824 | Output tokens: 1,224]

Tecnico 3


In [12]:
agent = IndustrialMaintenanceAgent()
question = agent.get_sample_questions()[1]
response = agent.query(question)
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Quais são os tipos de equipamento que passaram por manutenção nos últimos 3 meses?                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Calculate the date 3 months ago from today                                                                     
  three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)                                         
                                                                                                                   
  # Format the date to match the SQL query format                                                                  
  formatted_date = three_months_ago.strftime('%Y-%m-%d')                                                           
                                                                                                                   
  # Construct the SQL query                                                                                        
  query = f"SELECT DISTINCT tipo_equipamento FROM ordens_de_manutencao WHERE data >= '{formatted_date}';"          
                                                                                                                   
  # Execute the query and print the results                                                                        
  equipment_types = sql_maintenance_query(query=query)                                                             
  print(equipment_types)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Erro na consulta SQL: (sqlite3.OperationalError) no such table: ordens_de_manutencao
[SQL: SELECT DISTINCT tipo_equipamento FROM ordens_de_manutencao WHERE data >= '2025-04-02';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Out: None

[Step 1: Duration 3.23 seconds| Input tokens: 2,085 | Output tokens: 205]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(["Caldeira", "Bomba", "Motor", "Compressor"])                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ['Caldeira', 'Bomba', 'Motor', 'Compressor']

[Step 8: Duration 1.35 seconds| Input tokens: 27,406 | Output tokens: 1,243]

['Caldeira', 'Bomba', 'Motor', 'Compressor']


In [13]:
agent = IndustrialMaintenanceAgent()
question = str(input("Digite sua pergunta sobre a base de dados de manutenção: "))
response = agent.query(question)
print(response)
#Qual o nome do técnico que trabalhou em mais ordens de manutenção?

Digite sua pergunta sobre a base de dados de manutenção: Qual o nome do técnico que trabalhou em mais ordens de manutenção?


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Qual o nome do técnico que trabalhou em mais ordens de manutenção?                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = """                                                                                                      
  SELECT technician_name, COUNT(maintenance_order_id) as order_count                                               
  FROM technicians                                                                                                 
  JOIN maintenance_orders ON technicians.id = maintenance_orders.technician_id                                     
  GROUP BY technician_name                                                                                         
  ORDER BY order_count DESC                                                                                        
  LIMIT 1;                                                                                                         
  """                                                                                                              
  result = sql_maintenance_query(query=query)                                                                      
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Erro na consulta SQL: (sqlite3.OperationalError) no such table: technicians
[SQL: 
SELECT technician_name, COUNT(maintenance_order_id) as order_count
FROM technicians
JOIN maintenance_orders ON technicians.id = maintenance_orders.technician_id
GROUP BY technician_name
ORDER BY order_count DESC
LIMIT 1;
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

Out: None

...

[Step 8: Duration 2.50 seconds| Input tokens: 28,178 | Output tokens: 1,361]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: 
Root=1-686390a1-46bbbc787f62fef027f93418;6c844998-5cde-46fb-a140-b4620c7a317c)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 9: Duration 0.23 seconds]

Erro ao processar a pergunta: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-686390a1-46bbbc787f62fef027f93418;6c844998-5cde-46fb-a140-b4620c7a317c)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.


# SQL

In [ ]:

import os
import sqlite3
import pandas as pd

db_path = "/content/manutencao_industrial.db"

def get_database_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema = {}
    for table_name_tuple in tables:
        table_name = table_name_tuple[0]
        cursor.execute(f"PRAGMA table_info('{table_name}');")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]
        schema[table_name] = column_names
    conn.close()
    return schema

print("Esquema de la base de datos:")
db_schema = get_database_schema(db_path)
for table, columns in db_schema.items():
    print(f"Tabla: {table}, Columnas: {columns}")

Esquema de la base de datos:
Tabla: equipamentos, Columnas: ['id_equipamento', 'tipo', 'localizacao', 'status']
Tabla: ordens_manutencao, Columnas: ['id_ordem', 'id_equipamento', 'data_abertura', 'data_conclusao', 'tipo_manutencao', 'status']
Tabla: tecnicos, Columnas: ['id_tecnico', 'nome', 'especialidade', 'turno']
Tabla: ordem_tecnico, Columnas: ['id_ordem', 'id_tecnico']


In [23]:
conn = sqlite3.connect(db_path)
try:
    sql_query = """
    SELECT
      t.nome
    FROM
      tecnicos AS t
    JOIN
      ordem_tecnico AS ot
      ON t.id_tecnico = ot.id_tecnico
    WHERE
      ot.id_ordem = 32;
    """
    df_tecnico = pd.read_sql_query(sql_query, conn)

    # Print the result
    if not df_tecnico.empty:
        print("Técnico(s) que trabalhou(aram) na ordem de manutenção número 32:")
        print(df_tecnico)
    else:
        print("Nenhum técnico encontrado para a ordem de manutenção número 32.")

except Exception as e:
    print(f"Ocorreu um erro ao executar a consulta: {e}")

Técnico(s) que trabalhou(aram) na ordem de manutenção número 32:
        nome
0  Tecnico 3


In [24]:
try:
    # SQL query to find equipment types maintained in the last 3 months
    sql_query = """
    SELECT
      DISTINCT e.tipo
    FROM
      equipamentos AS e
    JOIN
      ordens_manutencao AS om
      ON e.id_equipamento = om.id_equipamento
    WHERE
      om.data_abertura >= date('now', '-3 months');
    """

    # Execute the query using pandas and store the result in a DataFrame
    df_tipos_equipamento = pd.read_sql_query(sql_query, conn)

    # Print the result
    if not df_tipos_equipamento.empty:
        print("Tipos de equipamento que passaram por manutenção nos últimos 3 meses:")
        for index, row in df_tipos_equipamento.iterrows():
            print(f"- {row['tipo']}")
    else:
        print("Nenhum tipo de equipamento encontrado com manutenção nos últimos 3 meses.")

except Exception as e:
    print(f"Ocorreu um erro ao executar a consulta: {e}")

Tipos de equipamento que passaram por manutenção nos últimos 3 meses:
- Caldeira
- Bomba
- Motor
- Compressor
